In [2]:
import pandas as pd 
import pymongo
import os 

## Connection to the Database

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.project2

## Importing all Datasets

In [4]:
csvpath = "Resources/food_data2.csv"
csvpath6 = "Resources/freedom.csv"
csvpath4 = "Resources/happiness_data/2018.csv"
csvpath5 = "Resources/happiness_data/2019.csv"

food2_data = pd.read_csv(csvpath, encoding= 'unicode_escape')
freedom = pd.read_csv(csvpath6, encoding= 'unicode_escape')
happiness_2018 = pd.read_csv(csvpath4, encoding= 'unicode_escape')
happiness_2019 = pd.read_csv(csvpath5, encoding= 'unicode_escape')

In [5]:
#Merging the two separate happiness datasets 
happiness_2019 = pd.merge(happiness_2018, happiness_2019,
                                 how='outer', on='Country or region')

## Cleaning the Data

## Importing into DataBase